In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

c:\Users\jaesc2\GitHub\skforecast


In [7]:
import pytest
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from skforecast.preprocessing import RollingFeatures
from skforecast.recursive import ForecasterRecursive
from skforecast.direct import ForecasterDirect
from skforecast.direct import ForecasterDirectMultiVariate

# Fixtures
# from skforecast.direct.tests.tests_forecaster_direct.fixtures_forecaster_direct import y
# from skforecast.direct.tests.tests_forecaster_direct.fixtures_forecaster_direct import exog, exog_predict
from skforecast.direct.tests.tests_forecaster_direct_multivariate.fixtures_forecaster_direct_multivariate import series
from skforecast.direct.tests.tests_forecaster_direct_multivariate.fixtures_forecaster_direct_multivariate import exog, exog_predict

In [13]:
forecaster = ForecasterDirectMultiVariate(
                     regressor = LinearRegression(),
                     level     = 'l1',
                     steps     = 5,
                     lags      = 5
                 )
forecaster.fit(series=series)

recommended_n_boot = 5
for k, v in forecaster.in_sample_residuals_by_bin_.items():
    forecaster.in_sample_residuals_by_bin_[k] = v[:recommended_n_boot]
    
results = forecaster.predict_bootstrapping(
    steps=5, n_boot=recommended_n_boot, use_in_sample_residuals=True, use_binned_residuals=True
)
results

,level,pred_boot_0,pred_boot_1,pred_boot_2,pred_boot_3,pred_boot_4
50,l1,0.989501,0.684815,0.881146,0.755644,0.978991
51,l1,0.752124,0.452307,0.625725,0.414657,0.264310
52,l1,0.306092,0.409670,0.425193,0.419100,0.383594
53,l1,0.036610,0.745491,0.289779,0.467800,0.607408
54,l1,0.486608,0.399265,0.645374,0.364456,0.097450


In [14]:
results.filter(regex='^pred_boo', axis=1).to_numpy()

array([[0.98950112, 0.68481511, 0.88114584, 0.7556436 , 0.97899051],
       [0.75212361, 0.45230689, 0.62572451, 0.41465675, 0.26430958],
       [0.30609163, 0.40967018, 0.42519328, 0.41910018, 0.38359395],
       [0.03661046, 0.74549056, 0.28977873, 0.46779981, 0.60740805],
       [0.48660788, 0.39926529, 0.64537401, 0.3644564 , 0.09744978]])